<a href="https://colab.research.google.com/github/Gredmond/StreamlitProjects/blob/main/Setiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import google.generativeai as genai
import enum
from typing_extensions import TypedDict
import json
from google.colab import userdata

print(userdata.get("BLUESKY_USER"))

dairtech.bsky.social


In [ ]:
class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"

class AnalysisResult(TypedDict):
    is_stock_related: bool
    sentiment: Sentiment

In [ ]:
search_term = 'Dutch Bros'
ticker = 'Dutch Bros'

In [ ]:
BLUESKY_HANDLE = userdata.get("BLUESKY_USER")
BLUESKY_PASSWORD = userdata.get("BLUESKY_CREDS")

In [ ]:


n = 100  # Number of latest posts to retrieve
# Authenticate and obtain access token
auth_response = requests.post(
    'https://bsky.social/xrpc/com.atproto.server.createSession',
    json={'identifier': BLUESKY_HANDLE, 'password': BLUESKY_PASSWORD}
)
auth_response.raise_for_status()
access_token = auth_response.json().get('accessJwt')

# Set up the request headers with the access token
headers = {'Authorization': f'Bearer {access_token}'}

# Define the search parameters
params = {
    'q': search_term,
    'sort': 'latest',
    'limit': n
}

# Perform the search request
search_response = requests.get(
    'https://bsky.social/xrpc/app.bsky.feed.searchPosts',
    headers=headers,
    params=params
)
search_response.raise_for_status()
posts = search_response.json().get('posts', [])

data = []
for post in posts:
    author = post.get('author', {}).get('handle', 'Unknown')
    content = post.get('record', {}).get('text', 'No content')
    created_at = post.get('record', {}).get('createdAt', 'Unknown date')
    data.append({'Date': created_at, 'Content': content, 'Author': author})



In [ ]:
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("models/gemini-2.0-flash") # gemini-2.0-flash-exp


In [ ]:
# Convert list of dictionaries to DataFrame
df = pd.DataFrame(data)
# Convert 'Date' column to datetime format for better handling
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Display the DataFrame
df[['Date','Content']].head()


,Date,Content
0,2025-02-28 03:13:37.789000+00:00,one time i was in the dutch bros drive thru an...
1,2025-02-28 02:17:28.770000+00:00,I learned about it from a Dutch Bros barista. ...
2,2025-02-28 02:05:03.280000+00:00,Dutch Bros opening new location in North Count...
3,2025-02-28 00:45:46.587000+00:00,Every Dutch Bros order I’ve ever seen/heard de...
4,2025-02-27 22:12:57.392000+00:00,I thought that was their warm up top sponsor l...


In [ ]:

def analyze_post(content: str) -> AnalysisResult:
    prompt = f"""
    company is stock ticker {ticker}:
    Analyze the following post and determine:
    1. Whether it is related to the company stock, or relates to or discusses past, current, or future stock performance of company explicitly
    or it relates to company's products in that case it is not stock related but should still be analyzed.
    2. Classify the sentiment as positive, negative, or neutral.
    Post: "{content}"
    """
    response = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=AnalysisResult
        )
    )
    if response.candidates:
        candidate_content = response.candidates[0].content
        result_text = ''.join(part.text for part in candidate_content.parts)
        try:
            result = json.loads(result_text)
            is_stock_related = result.get('is_stock_related')
            sentiment = result.get('sentiment')
            if is_stock_related is not None and sentiment is not None:
                return is_stock_related, sentiment
            else:
                # print("Missing expected keys in the response")
                return None, None
        except json.JSONDecodeError:
            print("Failed to decode JSON response")
            return None, None
    else:
        print("No candidates returned")
        return None, None
# Apply the analysis to each post
df[['is_stock_related', 'sentiment']] = df['Content'].apply(
    lambda x: pd.Series(analyze_post(x))
)

In [ ]:
df

,Date,Content,Author,is_stock_related,sentiment
0,2025-02-27 22:12:57.392000+00:00,I thought that was their warm up top sponsor l...,munchkinfunk.bsky.social,True,neutral
1,2025-02-27 21:13:08.398000+00:00,LOVE Dutch Bros! Looks so tasty!,amyhoneyjellybee.bsky.social,None,None
2,2025-02-27 20:29:16.706000+00:00,You could never satisfy me as much as one sip ...,goddessocelot.bsky.social,None,None
3,2025-02-27 17:39:17.577000+00:00,Why are most of Dutch Bros flavored lattes mad...,funkyskeletons.bsky.social,None,None
4,2025-02-27 05:22:54.096000+00:00,Waited until the LAST DAY possible to redeem a...,dennis.pascual.co,None,None
...,...,...,...,...,...
95,2025-02-20 23:14:16.199000+00:00,Mmm love mixing my Adderall with Dutch Bros an...,negaini.bsky.social,None,None
96,2025-02-20 18:10:33.236000+00:00,I have been a Dutch Bros fanatic these last fe...,casslo7.bsky.social,None,None
97,2025-02-20 18:05:13.878000+00:00,Way to go! I made the same switch last year & ...,god-of-sassgard.bsky.social,None,None
98,2025-02-20 17:49:49.667000+00:00,love me a dutch bros sticker drop,gamerbeeb.bsky.social,None,None


In [ ]:
filtered_df = df[df['sentiment'].isin(['positive', 'negative'])]
filtered_df



In [ ]:
# prompt: From filtered_df I would like the sum of 'positive' and 'negative' rows for each day

# Group by day and sentiment, then sum
daily_sentiment_sum1 = df.groupby(df['Date'].dt.date)['sentiment'].value_counts().unstack(fill_value=0)

# Display the result
daily_sentiment_sum1


sentiment,negative,neutral,positive
Date,,,
2025-02-20,0,1,0
2025-02-21,0,7,2
2025-02-22,3,10,5
2025-02-23,3,6,5
2025-02-24,2,7,6
2025-02-25,1,5,4
2025-02-26,1,9,8
2025-02-27,0,3,3
2025-02-28,1,1,2


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=daily_sentiment_sum1)

https://docs.google.com/spreadsheets/d/11V151Vc4OPiia-edf4tclPjndLRQsTM6sKAh8GNp5nU#gid=0
